In [1]:
import pdfplumber
import fitz
import camelot
import warnings , math, collections , os, re
import pickle
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning) 

In [2]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

tata_path = path + r"\files\TataFactSheet2024.pdf"

#dry run paths
dry_run_path = path + r"\output\DryRun.pdf"

In [3]:
#TATA PATHS
#tata output path
no_image_path = path +r"\output\tata\TatanoImgPdf.pdf"
textual_pdf_path = path + r"\output\tata\TatatextalPdf.pdf"
tabular_pdf_path = path + r"\output\tata\TatatabularPdf.pdf"

#pickkle data paths tata
pickle_text = r"\output\pkl\tata\textual_data.pkl"
pickle_tab = r"\output\pkl\tata\tabular_data.pkl"
pickle_nonimg = r"\output\pkl\tata\nonimg_data.pkl"
pickle_all = r'\output\pkl\tata\all_data.pkl'

In [4]:
with open(path + pickle_all , 'rb') as file:
    document_data = pickle.load(file)  

In [5]:
document_data['total_pages']

109

In [355]:
pdf = fitz.open(tata_path)

for pgn in range(pdf.page_count):
    page = pdf[pgn]
    
    text_data = page.get_text('dict')['blocks']
    sorted_text_data = sorted(text_data, key=lambda x :(x['bbox'][1],x['bbox'][0]))
    
    first_block = sorted_text_data[0]
    if "lines" in first_block:
        for line in first_block['lines']:
            for span in line['spans']:
                if re.match(r'^(tata|samco|icici).*fund$', span['text'], re.IGNORECASE):
                    print(pgn, span['text'])

16 Tata Large Cap Fund
17 Tata Flexi Cap Fund
18 Tata Large & Mid Cap Fund
19 Tata Equity P/E Fund
20 Tata Mid Cap Growth Fund
21 Tata ELSS Tax Saver Fund
22 Tata Small Cap Fund
23 Tata Focused Equity Fund
24 Tata Nifty 50 Index Fund
25 Tata Nifty Auto Index Fund
26 Tata Nifty India Tourism Index Fund
27 Tata Nifty Financial Services Index Fund
28 Tata Nifty MidSmall Healthcare Index Fund
29 Tata Nifty Realty Index Fund
30 TATA Nifty500 Multicap India Manufacturing 50:30:20 Index Fund
31 Tata Nifty500 Multicap Infrastructure 50:30:20 Index Fund
32 Tata BSE Sensex Index Fund
33 Tata Nifty 50 Exchange Traded Fund
34 Tata Nifty Private Bank Exchange Traded Fund
35 Tata Dividend Yield Fund
36 Tata Nifty India Digital ETF Fund of Fund
37 Tata Nifty India Digital Exchange Traded Fund
38 Tata Silver Exchange Traded Fund
39 Tata Silver ETF Fund of Fund
40 Tata Gold Exchange Traded Fund
41 Tata Gold ETF Fund of Fund
42 Tata Nifty Midcap 150 Momentum 50 Index Fund
43 Tata Nifty200 Alpha 30 Index

In [17]:

title_data = dict()
for pgn,page in enumerate(document_data['blocks_data']):
    sortedPage = sorted(page['blocks'], key= lambda k: (round(k['bbox'][1]), round(k['bbox'][0]))) #sort t to b, l to r
    firstBlock = sortedPage[0] #get first block of sorted data
    
    if 'lines' in firstBlock:
        for line in firstBlock['lines']:
            for span in line['spans']:
                text = span['text'].lower()
                
                #regex condition
                cond1 = re.findall('^samco|^tata|fund$', text)
                if cond1:
                    
                    title_data[pgn] = text
                         

In [ ]:
pages = list(title_data.keys())
pages

In [153]:
bbox = (15,55,170,615)

In [161]:
def get_data_clipped(input:str, pageSelect:list, bbox:set):   
    document = fitz.open(input)
    finalData = []
    
    for pgn, title in pageSelect.items():
        page = document[pgn]
        
        blocks = page.get_text('dict', clip = bbox)['blocks']
        sorted_blocks = sorted(blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
        
        finalData.append(
            {
                'page':pgn,
                'title': title,
                "block": sorted_blocks
            }
        )          
    return finalData

def remove_keys(data:list, keys_to_remove:set):
    if isinstance(data, list):
        # Process each element in the list
        return [remove_keys(item, keys_to_remove) for item in data]
    elif isinstance(data, dict):
        # Process each key-value pair in the dictionary
        return {key: remove_keys(value, keys_to_remove) for key, value in data.items() if key not in keys_to_remove}
    else:
        # Return data as is if it's neither a dict nor a list
        return data
    
def clean_text(input_str):
    # Step 1: Remove extra spaces between words
    input_str = re.sub(r'\s+', ' ', input_str.strip())
    # Step 2: Fix split words
    #input_str = re.sub(r'(\b[A-Z]+)\s+([A-Z]+\b)', r'\1\2', input_str)
    # Step 3: Remove trailing special characters like '/', '**', '^'
    input_str = re.sub(r'[/*^]+$', '', input_str)
    return input_str

In [316]:
data = get_data_clipped(tata_path, title_data, bbox)

In [ ]:
data[0]['block']

In [334]:
sub_headers = set()
sizes = set()
remove_text = [':', "  "]
patterns_text = [r'^MULTI|(MENT|TORS)$']
for page in data:
    for blocks in page['block']:
        if 'lines' in blocks:
            
            for line in blocks['lines']:
                
                line['spans'] = [
                    span for span in line['spans']
                    if not any(re.match(pattern, span['text'].strip()) for pattern in patterns_text)
                ]
                
                # Filter out spans with text that is whitespace or empty
                line['spans'] = [
                    span for span in line['spans']
                    if span['text'].strip()  # Keep spans with non-whitespace text
                ]
            

                for span in line['spans']:
                    text = clean_text(span['text'].strip())
                    
                    #imp conditions for text
                    conditions = [span['size'] == 6 ,
                                span['font'] == 'Swiss721BT-BoldCondensed',
                                span['color']== -15570765]
                    if all(conditions):
        
                        sub_headers.add(text)
                        span['text'] = text
                        span['size'] = 30.0
                    
                    sizes.add(span['size'])

In [342]:
for cleaned_data in data:
    for block in cleaned_data.get("block", []):
        for line in block.get("lines", []):
            combined_spans = []
            for span in line.get("spans", []):
                if combined_spans and all(
                    combined_spans[-1].get(key) == span.get(key)
                    for key in ["flags", "size", "color"]
                ):
                    # Combine text if spans are similar
                    combined_spans[-1]["text"] += " " + span["text"]
                else:
                    combined_spans.append(span)
            line["spans"] = combined_spans

In [ ]:
data[0]['block']

In [314]:
keys_to_remove = {} #{'ascender', 'descender', 'bbox','wmode','dir','number'}
cleaned_data = remove_keys(data, keys_to_remove)

In [ ]:
data[1]['block']

In [337]:
def create_matrix_structure(data: list, subheader_font: float, content_font: float):
    # extract unique coordinates
    coordinates = []
    fonts = set()

    for block in data['block']:
        for line in block['lines']:
            for span in line['spans']:
                origin = tuple(span['origin'])  # Extract (x, y) coordinates from origin
                coordinates.append(origin)
                fonts.add(span['size'])

    coordinates = sorted(set(coordinates), key=lambda c: (c[1], c[0]))  # Sort by y, then x
    fonts = sorted(fonts)

    # Step 2: Initialize the matrix
    coord_to_index = {coord: idx for idx, coord in enumerate(coordinates)}
    font_to_index = {font: idx for idx, font in enumerate(fonts)}
    matrix = np.zeros((len(coordinates), len(fonts)), dtype=object)

    # Step 3: Fill the matrix
    for block in data['block']:
        for line in block['lines']:
            for span in line['spans']:
                origin = tuple(span['origin'])  # Get (x, y) coordinates from origin
                font = span['size']
                if origin in coord_to_index and font in font_to_index:
                    row = coord_to_index[origin]
                    col = font_to_index[font]
                    if matrix[row, col] == 0:
                        matrix[row, col] = []
                    matrix[row, col].append(span)  # Append the entire span dictionary

    # Step 4: Generate the nested dictionary
    nested_dict = {}
    current_subheader = None

    for row_idx, coord in enumerate(coordinates):
        for col_idx, font in enumerate(fonts):
            if matrix[row_idx, col_idx] != 0:
                spans = matrix[row_idx, col_idx]

                for span in spans:
                    if font == subheader_font:
                        current_subheader = span
                        nested_dict[current_subheader['text']] = []
                    elif font <= content_font and current_subheader:
                        nested_dict[current_subheader['text']].append(span)

    return nested_dict
def generate_pdf_from_data(data:list, output_path:str):
    """
    Generates a PDF from the nested dictionary data structure.

    Parameters:
        data (dict): The nested dictionary containing sections and fitz spans.
        output_path (str): The file path where the PDF will be saved.
    """
    # Create a new PDF document
    pdf_document = fitz.open()
    
    for section, spans in data.items():
        # Add a new page for each section
        page = pdf_document.new_page()
        text_position = 72  # Initial vertical position (used only for section titles)

        # Add section title
        title_font_size = 14
        page.insert_text(
            (72, text_position),
            section,
            fontsize=title_font_size,
            fontname="helv",
            color=(0, 0, 1),
        )

        # Iterate through each span in the section
        for span in spans:
            bbox = span.get("bbox", [0, 0, 0, 0])  # Use bbox for exact placement

            # Error handling for font issues
            try:
                page.insert_text(
                    (bbox[0], bbox[1]),  # Use bbox coordinates for exact placement
                    span["text"],
                    fontsize=span["size"],
                    fontname="helv",
                    color=tuple(int(span["color"] & 0xFFFFFF) for _ in range(3)),  # Convert span color
                )
            except Exception:
                page.insert_text(
                    (bbox[0], bbox[1]),  # Use bbox coordinates for exact placement
                    span["text"],
                    fontsize=span["size"],
                    fontname="helv",
                    color=(1, 0, 0),  # Fallback color for errors
                )

    # Save the created PDF
    pdf_document.save(output_path)
    pdf_document.close()
    print(f"PDF successfully generated and saved to: {output_path}")

In [ ]:
matrix_data = create_matrix_structure(data[1],30,10)

In [327]:
generate_pdf_from_data(matrix_data, path + r"\output\samcoDryRun.pdf")

PDF successfully generated and saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\samcoDryRun.pdf


In [343]:
with pdfplumber.open(path +r"\output\samcoDryRun.pdf") as pdf:
    final_data = []
    final_data_generated = {}
    
    for page in pdf.pages:
        # extract text from the page
        text = page.extract_text()
        final_data.append(text)

In [344]:
final_data

['INVESTMENT STYLE\nPrimarily focuses on investing in equity and equity related\ninstruments of well researched companies across market\ncapitalization.',
 'INVESTMENT OBJECTIVE\nThe investment objective of the Scheme is to generate capital\nappreciation over medium to long term. However, there is no\nassurance or guarantee that the investment objective of the Scheme\nwill be achieved. The scheme does not assure or guarantee any\nreturns.',
 'DATE OF ALLOTMENT\nSeptember 06,2018',
 'FUND MANAGER\nAmey Sathe (Managing Since 05-Apr-23 and overall\nexperience of 16 years)',
 'ASSISTANT FUND MANAGER\nAditya Bagul (Managing Since 03-Oct-23 and overall\nexperience of 11 years)',
 'BENCHMARK\nStd. Dev (Annualised) 12.06 13.50\nSharpe Ratio 0.54 0.73\nPortfolio Beta 0.82 NA\nR Squared 0.90 NA\nTreynor 0.67 NA\nJenson -0.13 NA\n^Risk-free rate based on the FBIL Overnight MIBOR rate of 6.76% as\non Nov 29, 2024\nFor calculation methodology please refer to Pg 100.',
 'NAV (in Rs.)\nDirect - IDCW 